<a href="https://colab.research.google.com/github/ronidad/Moring-week-4IP/blob/main/Ronald_Mosomi_week_4_IP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Paris electric cars (Blue Cars) analysis

# Import required libraries

In [192]:
import pandas as pd
import matplotlib.pyplot as plt


# Read data required 

In [193]:
df = pd.read_csv("data.csv")

# Data cleaning

In [194]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 25 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Address              5000 non-null   object
 1   Cars                 5000 non-null   int64 
 2   Bluecar counter      5000 non-null   int64 
 3   Utilib counter       5000 non-null   int64 
 4   Utilib 1.4 counter   5000 non-null   int64 
 5   Charge Slots         5000 non-null   int64 
 6   Charging Status      5000 non-null   object
 7   City                 5000 non-null   object
 8   Displayed comment    111 non-null    object
 9   ID                   5000 non-null   object
 10  Kind                 5000 non-null   object
 11  Geo point            5000 non-null   object
 12  Postal code          5000 non-null   int64 
 13  Public name          5000 non-null   object
 14  Rental status        5000 non-null   object
 15  Scheduled at         47 non-null     object
 16  Slots 

In [195]:
df.isnull().sum()

Address                   0
Cars                      0
Bluecar counter           0
Utilib counter            0
Utilib 1.4 counter        0
Charge Slots              0
Charging Status           0
City                      0
Displayed comment      4889
ID                        0
Kind                      0
Geo point                 0
Postal code               0
Public name               0
Rental status             0
Scheduled at           4953
Slots                     0
Station type              0
Status                    0
Subscription status       0
year                      0
month                     0
day                       0
hour                      0
minute                    0
dtype: int64

In [196]:
#Validity 

#Drop columns with many null values

df.drop(columns=['Displayed comment','Scheduled at'],inplace=True)



In [197]:
#Checking whether there are duplicates to drop

#There is no duplicate

df.duplicated().any()

False

In [198]:
#Uniformity
df.columns = df.columns.str.strip().str.replace(" ", "_").str.title()
#Remove spaces in columns, replace spaces with _ and make all of the title case


In [199]:
df.columns

Index(['Address', 'Cars', 'Bluecar_Counter', 'Utilib_Counter',
       'Utilib_1.4_Counter', 'Charge_Slots', 'Charging_Status', 'City', 'Id',
       'Kind', 'Geo_Point', 'Postal_Code', 'Public_Name', 'Rental_Status',
       'Slots', 'Station_Type', 'Status', 'Subscription_Status', 'Year',
       'Month', 'Day', 'Hour', 'Minute'],
      dtype='object')

In [200]:
#Create dates column by joining the year, month, day, hour and minute to enable us 
df['Dates'] = pd.to_datetime(df[['Year', 'Month', 'Day', 'Hour', 'Minute']])
autolib = df.sort_values(by='Dates')

In [201]:
#Drop unwanted columns 
autolib.drop(columns=['Address','Cars','Id','Geo_Point','Year','Month','Day','Minute','Station_Type'],inplace=True)

In [202]:
#autolib.groupby(['Dates','Public Name']).sum().sort_values(['Public Name', 'Dates'], ascending=[True, True]).head()
autolib_active = autolib.groupby(['Dates','Public_Name','City']).sum().sort_values(['Public_Name', 'Dates'], ascending=[True, True])

#This is helping us sort all stations(public name) and to ensure that we get each grouped ondividually 

In [203]:
autolib_active.head()

,,,Bluecar_Counter,Utilib_Counter,Utilib_1.4_Counter,Charge_Slots,Postal_Code,Slots,Hour
Dates,Public_Name,City,,,,,,,
2018-04-02 19:28:00,Alfortville/Blanqui/61,Alfortville,0,0,0,0,94140,0,19
2018-04-04 09:05:00,Alfortville/Blanqui/61,Alfortville,0,0,0,0,94140,0,9
2018-04-07 18:40:00,Alfortville/Blanqui/61,Alfortville,0,0,0,0,94140,0,18
2018-04-09 09:01:00,Alfortville/Blanqui/61,Alfortville,0,0,0,0,94140,0,9
2018-04-01 00:36:00,Alfortville/Charles de Gaulle/16,Alfortville,2,0,0,0,94140,0,0


In [204]:
#Reset the indeces so that we can easily use the dataframe
autolib_active.reset_index(inplace=True)

In [205]:
#Add boolean station match column to help us klnow if the current station we are calculating the difference is the same as previous one
autolib_active['Station_Match'] = autolib_active.Public_Name.eq(autolib_active.Public_Name.shift())

In [206]:
 #Calculate the values of the current difference from the previous column using shifty
 autolib_active['Blue_Difference']= autolib_active.Bluecar_Counter.shift(1) - autolib_active.Bluecar_Counter

In [207]:
autolib_active['All_Cars_Counter'] = autolib_active['Utilib_Counter'] + autolib_active['Utilib_1.4_Counter']
autolib_active['All_Difference']= autolib_active.All_Cars_Counter.shift(1) - autolib_active.All_Cars_Counter
autolib_active.loc[autolib_active['Station_Match'] == False, 'All_Difference'] = autolib_active.All_Cars_Counter*-1
#Update the colums where the previous columns in public name is not equla to current public name to have their cars equal to the one we found initially

autolib_active.loc[autolib_active['Station_Match'] == False, 'Blue_Difference'] = autolib_active.Bluecar_Counter*-1

#autolib_active['Kind'] = df['Kind']


In [208]:
autolib_active.head(50)

,Dates,Public_Name,City,Bluecar_Counter,Utilib_Counter,Utilib_1.4_Counter,Charge_Slots,Postal_Code,Slots,Hour,Station_Match,Blue_Difference,All_Cars_Counter,All_Difference
0,2018-04-02 19:28:00,Alfortville/Blanqui/61,Alfortville,0,0,0,0,94140,0,19,False,0.0,0,0.0
1,2018-04-04 09:05:00,Alfortville/Blanqui/61,Alfortville,0,0,0,0,94140,0,9,True,0.0,0,0.0
2,2018-04-07 18:40:00,Alfortville/Blanqui/61,Alfortville,0,0,0,0,94140,0,18,True,0.0,0,0.0
3,2018-04-09 09:01:00,Alfortville/Blanqui/61,Alfortville,0,0,0,0,94140,0,9,True,0.0,0,0.0
4,2018-04-01 00:36:00,Alfortville/Charles de Gaulle/16,Alfortville,2,0,0,0,94140,0,0,False,-2.0,0,0.0
5,2018-04-01 01:36:00,Alfortville/Charles de Gaulle/16,Alfortville,4,0,0,0,94140,0,1,True,-2.0,0,0.0
6,2018-04-03 04:03:00,Alfortville/Charles de Gaulle/16,Alfortville,3,0,0,0,94140,3,4,True,1.0,0,0.0
7,2018-04-03 22:27:00,Alfortville/Charles de Gaulle/16,Alfortville,4,0,0,0,94140,1,22,True,-1.0,0,0.0
8,2018-04-04 03:30:00,Alfortville/Charles de Gaulle/16,Alfortville,6,0,0,0,94140,0,3,True,-2.0,0,0.0
9,2018-04-04 09:40:00,Alfortville/Etienne Dolet/174,Alfortville,1,0,0,0,94140,5,9,False,-1.0,0,0.0


# Answering Questions

***Identify the most popular hour of the day for picking up a shared electric car (Bluecar) in the city of Paris over the month of April 2018.***

In [209]:
#where difference is negative it means vehicles have been retruned from the previous time there was a counter (Checking) since we have many vehicles: (Returning)
#Positivee means vehiles have been taken since we have fewer vehicles in the parking than the previous check: (Picking)


# Hour 21 is when Blue Cars are picked

#autolib_active[(autolib_active['Blue_Difference']>0) ].groupby('Hour').sum()['Blue_Difference'].sort_values(ascending =False).head()
autolib_active[(autolib_active['Blue_Difference']>0)&(autolib_active.City == "Paris") ].groupby(['Hour']).sum()['Blue_Difference'].sort_values(ascending =False).head()

Hour
21    109.0
18     91.0
22     88.0
17     88.0
10     81.0
Name: Blue_Difference, dtype: float64

***Most popular Hour for picking all other cars***

In [210]:
autolib_active[(autolib_active['All_Difference']>0) &(autolib_active.City == "Paris")].groupby('Hour').sum()['All_Difference'].sort_values(ascending =False).head()
#Hour 11 is the most popular for picking other cars 

Hour
11    20.0
12    18.0
18    17.0
3     16.0
23    16.0
Name: All_Difference, dtype: float64

***Most popular hour for returning Blue cars***

In [211]:


#Hour 9 is the most popular hour for returning blue cars
autolib_active[(autolib_active['Blue_Difference']<0) &(autolib_active.City == "Paris")].groupby('Hour').sum()['Blue_Difference'].sort_values().head()

Hour
9    -147.0
12   -141.0
6    -137.0
10   -129.0
21   -127.0
Name: Blue_Difference, dtype: float64

***What is the most popular hour for returning cars (other cars)?***

In [212]:
autolib_active[(autolib_active['All_Difference']<0) &(autolib_active.City == "Paris")].groupby('Hour').sum()['All_Difference'].sort_values().head()
# Hour 21 is the most popular hour for returning all cars 

Hour
6    -27.0
17   -24.0
21   -23.0
8    -22.0
5    -19.0
Name: All_Difference, dtype: float64

***What station is the most popular Overrally?***

In [213]:
df[(df['Kind']=="STATION") &(df.City == "Paris")]['Public_Name'].value_counts().head()

#The most popular station is Paris/Porte de Montrouge/8 

Paris/Porte de Montrouge/8    13
Paris/Philippe Auguste/126    11
Paris/Voltaire/182            11
Paris/Courcelles/69           11
Paris/Mathis/35               11
Name: Public_Name, dtype: int64

***Most popular station during popular picking hours***

In [222]:
df[(df['Kind']=="STATION") & (df['Hour']==5)]['Public_Name'].value_counts().head(1)
# the sttaion is Paris/ChÃ¢teau Landon/24  

Paris/ChÃ¢teau Landon/24    3
Name: Public_Name, dtype: int64

***What postal code is the most popular for picking up Blue cars? Does the most popular station belong to that postal code?***


In [215]:
autolib_active[(autolib_active['Blue_Difference']>0) &(autolib_active.City == "Paris")]['Postal_Code'].value_counts(ascending = False).head()

#The most popular postal code is 75015. It does not belong to the most popular station since the most popular station code is 75014

75015    76
75017    74
75016    70
75011    53
75013    52
Name: Postal_Code, dtype: int64

***Most popular postal code at the most popular picking hour***

In [216]:
autolib_active[(autolib_active['Blue_Difference']>0) & (df['Hour']==21) &(autolib_active.City == "Paris") ]['Postal_Code'].value_counts(ascending = False).head()
# Codes is 75017


75017    9
75015    4
75013    4
75019    3
75011    3
Name: Postal_Code, dtype: int64

***Most popular postal code for other cars***

In [217]:
autolib_active[(autolib_active['All_Difference']>0) &(autolib_active.City == "Paris")]['Postal_Code'].value_counts(ascending = False).head()
#Most popular postal code is 75017. 


75017    33
75015    28
75016    22
75020    20
75013    19
Name: Postal_Code, dtype: int64

In [218]:
autolib_active[(autolib_active['All_Difference']>0) & (df['Hour']==18) &(autolib_active.City == "Paris") ]['Postal_Code'].value_counts(ascending = False).head()

75017    3
75001    1
75012    1
75020    1
75008    1
Name: Postal_Code, dtype: int64

In [219]:
#Most popular postal code is 75017

In [220]:
#Most popular postal code overrally 
autolib_active[autolib_active['City']=='Paris']['Postal_Code'].value_counts().sort_values(ascending=False).head()

75015    274
75016    236
75017    221
75011    180
75020    169
Name: Postal_Code, dtype: int64

In [221]:
#The codes are slightly different overrally 